# Le Debruiteur
* Jonas Freiburghaus
* Romain Capocasale
* He-Arc, INF3dlm-a
* Image Processing course
* 2019-2020

# Import

In [ ]:
%matplotlib inline

from debruiteur.generator.datagenerator import DataGenerator
from debruiteur.models.models import build_conv_autoencoder
from debruiteur.noise.noise import *
from debruiteur.plots.plots import plot_model_loss
from debruiteur.preprocessing.preprocessor import *
from debruiteur.utils.utils import save_model, split_train_val_df

import cv2
import os
from skimage import measure

# Configurations

In [ ]:
noise_class_list = [
    GaussianNoise(mean=0, std=10),
    PoissonNoise(),
    UniformNoise(amplitude=100),
    SaltPepperNoise(freq=0.3),
    SquareMaskNoise(mask_shape=(10, 10), freq=0.1),
    SpeckleNoise(),
    AveragingBlurNoise(),
    GaussianBlurNoise(),
    MedianBlurNoise()
]

# Create Dataframe

In [ ]:
working_dir = os.path.abspath(os.getcwd())

df_original = make_original_dataframe(os.path.join(working_dir, "images"))
df_resized = make_resized_dataframe(df_original, os.path.join(working_dir, "resized_images"))
df_noised = make_noised_dataframe(df_resized, noise_class_list, os.path.join(working_dir, "noised_images"))
df_train, df_val = split_train_val_df(df_noised)

# Models training

## Data generator

In [ ]:
train_gen = DataGenerator(df_train)
val_gen = DataGenerator(df_val)

## Autoencoder

In [ ]:
ae_model = build_conv_autoencoder()
ae_model.summary()

In [ ]:
history = ae_model.fit_generator(train_gen, validation_data=val_gen, epochs=10)
plot_model_loss(history)

In [ ]:
save_model(ae_model, os.path.join(working_dir, "models"), "conv_autoencoder.h5")